In [2]:
# Imports

In [3]:
# Load X and Y from h5 files
def read_data_from_h5_files(x_path, y_path, labels_file):
    '''
        Read saved h5 files, shuffle, write the dataset split image ids in files
    '''
    # Load data H5 files
    hf1 = h5py.File(x_path, 'r+')
    X_train = hf1['X_train'].value
    hf1.close()
    hf2 = h5py.File(y_path, 'r+')
    Y_train = hf2['Y_train'].value
    hf2.close()
    print('Data loaded from h5 files')

    idxs = np.arange(X_train.shape[0])
    np.random.shuffle(idxs)
    X_train = X_train[idxs]
    Y_train = Y_train[idxs]
    print('Data shuffled')

    return X_train, Y_train

In [1]:
def vgg16_copy(IMAGE_SIZE, WEIGHT_FILE, N_class, LR, loss, OPT):
    '''
        Build/Define  VGG16 and load pretrained weights. used for all experiments
        with regularisation, dropout etc...
    '''
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
    img_input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
  
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    M = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = GlobalAveragePooling2D()(M)
    model = Model(img_input, x)
    model.load_weights(WEIGHT_FILE)
    F = Flatten(name='flatten')(M)
    FC1 = Dense(4096, activation='relu', name='fc1')(F)
    D1 = Dropout(0.5)(FC1)
    FC2 = Dense(500, activation='relu', name='fc2')(D1)
    D2 = Dropout(0.5)(FC2)
    y = Dense(N_class, activation='softmax', name='predictions')(x)
    model = Model(img_input, y)

    # Compile model with above settings
    model.compile(loss=LOSS, optimizer=OPT, metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# Configurations for the training
n_class = 2
batch_size = 50
epochs = 30
image_size = 224

# More advanced configurations
learning_rate = 0.01
optimizer = 'adadelta'
validation_split = 0.2

# Data directories
x_train_path = ''
y_train_path = ''
#labels_file = path.join(ds_base_dir, 'subsets/'+CONFIG['dataset_name']+'.txt')

In [ ]:
X_train, Y_train = read_data_from_h5_files(x_train_path, y_train_path, labels_file)
model = model_CPCPDHDH(image_size, N_class)
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=val_split)

In [ ]:
model.save(model_loc)
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
train_acc = history.history['acc']

# Display the Graph of Epochs vs Error